In [6]:
import sys
import config
from config import *
from bs4 import BeautifulSoup
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import numpy as np
import pickle
from IPython.display import HTML

In [2]:
data = pd.read_csv('data_clusters.csv')

In [3]:
from config import *
client_credentials_manager = SpotifyClientCredentials(client_id=client_ID, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_ID,
                                                           client_secret=client_secret))

In [ ]:
def recommend_songs():


    # Initialize Spotify API client
    client_credentials_manager = SpotifyClientCredentials(client_id=client_ID, client_secret=client_secret)
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_ID, client_secret=client_secret))

    while True:
        # Get input from the user
        artist = input("Type in the artist (or type 'exit' to stop): ")

        if artist.lower() == 'exit':
            print("Exiting the program.")
            break

        title = input("Type in the song name: ")    
            
        def search_song(title, artist, limit):
            try:
                results = sp.search(q=f"{title} {artist}", type='track', limit=limit)
                song_id = results['tracks']['items'][0]['id']
                return song_id
            except IndexError:
                print("Song not found!")
                return ""
    
        # Search for the song
        song_id = search_song(title, artist, 5)
    
        # Get audio features
        audio_features = sp.audio_features([song_id])
        song = pd.DataFrame(audio_features)
    
        # Data preprocessing
        features = song.select_dtypes(exclude=[object])
        columns_to_drop = ['liveness', 'duration_ms', 'time_signature']
        features.drop(columns=columns_to_drop, inplace=True)
    
        # Load scaler
        scaler_filename = "./feature_scaler.pickle"
        with open(scaler_filename, "rb") as file:
            scaler = pickle.load(file)
    
        # Transform audio features
        user_song_audio_features = scaler.transform(features)
        user_song_audio_features_df = pd.DataFrame(user_song_audio_features, columns=features.columns)
    
        # Load UMAP model
        umap_filename = "./umap.pickle"
        with open(umap_filename, "rb") as file:
            umap = pickle.load(file)
    
        # Transform with UMAP
        user_song_umap = umap.transform(user_song_audio_features_df)
        user_song_umap_df = pd.DataFrame(user_song_umap, columns=["UMAP_1", "UMAP_2"])
    
        # Load KMeans model
        cluster_model_filename = "./kmeans.pickle"
        with open(cluster_model_filename, "rb") as file:
            model = pickle.load(file)
    
        # Predict cluster
        user_song_cluster = model.predict(user_song_umap_df)[0]
    
        if song_id in data[data['hot_song'] == 1]['id'].tolist():
            result = data[(data['hot_song'] == 1) & (data['cluster'] == user_song_cluster)].sample(5)
        else:
            result = data[(data['hot_song'] != 1) & (data['cluster'] == user_song_cluster)].sample(5)
    
        result = pd.DataFrame(result)
        result['Spotify_Link'] = 'https://open.spotify.com/track/' + result['id']
        HTML(result[['Spotify_Link']].to_html(escape=False))
        output = result[['Title', 'Artists', 'Spotify_Link']]
        pd.set_option('display.max_colwidth', None)
        print( )
        print('Here are songs you may like:')
        display(output)
        column_content = output['Spotify_Link'].tolist()
        for content in column_content:
            print(content)
# Call the function
recommend_songs()


Type in the artist (or type 'exit' to stop):  cake
Type in the song name:  by the ocean



Here are songs you may like:


,Title,Artists,Spotify_Link
120,Get You,Daniel Caesar,https://open.spotify.com/track/7zFXmv6vqI4qOt4yGf3jYZ
1438,22 (OVER S∞∞N),Bon Iver,https://open.spotify.com/track/3EYj5rxRbNM3nijF9dOtry
230,Full Circle,Big Sean,https://open.spotify.com/track/4Na2Xo5yQCr5ucwfg8EGIQ
714,Do You Mind,DJ Khaled,https://open.spotify.com/track/4VJ7mnW4wjjIgPBM6KWvPu
2357,Christmas Time Is Here,Vince Guaraldi Trio,https://open.spotify.com/track/3bMfUUpzUzqWp9II00yRhE


https://open.spotify.com/track/7zFXmv6vqI4qOt4yGf3jYZ
https://open.spotify.com/track/3EYj5rxRbNM3nijF9dOtry
https://open.spotify.com/track/4Na2Xo5yQCr5ucwfg8EGIQ
https://open.spotify.com/track/4VJ7mnW4wjjIgPBM6KWvPu
https://open.spotify.com/track/3bMfUUpzUzqWp9II00yRhE
